In [ ]:
# Package imports
import numpy as np
import sklearn
import sklearn.linear_model
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import matplotlib.pyplot as plt
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

%matplotlib inline

np.random.seed(1)


## 1. Loading the dataset and preprocessing

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Vehicles.csv to Vehicles.csv


In [ ]:
import pandas as pd
df = pd.read_csv('Vehicles.csv')

In [ ]:
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,y
0,100,36,73,199,73,6,162,40,20,127,189,401,125,72,6,19,200,204,bus
1,91,36,72,162,60,8,150,44,19,133,166,334,121,63,2,22,196,205,saab
2,91,41,64,148,61,8,129,51,18,142,161,249,153,68,6,12,194,201,van
3,86,39,58,125,55,5,117,57,17,134,140,204,148,69,7,6,190,194,van
4,95,53,95,202,65,10,193,34,22,160,220,559,237,71,3,2,188,196,saab


In [ ]:
yle = LabelEncoder()
df['y'] = yle.fit_transform(df['y'])
df.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,y
0,100,36,73,199,73,6,162,40,20,127,189,401,125,72,6,19,200,204,0
1,91,36,72,162,60,8,150,44,19,133,166,334,121,63,2,22,196,205,2
2,91,41,64,148,61,8,129,51,18,142,161,249,153,68,6,12,194,201,3
3,86,39,58,125,55,5,117,57,17,134,140,204,148,69,7,6,190,194,3
4,95,53,95,202,65,10,193,34,22,160,220,559,237,71,3,2,188,196,2


In [ ]:
df = df.values

In [ ]:
X = df[:,0:18].astype(np.int64)
y = df[:,18]



In [ ]:
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(y)

In [ ]:
x_data, test_data, y_data, test_labels = train_test_split(X, Y, test_size =0.15, random_state=0)
train_data, val_data, train_labels, val_labels = train_test_split(x_data, y_data, test_size =0.15, random_state=0)

In [ ]:
scaler = preprocessing.StandardScaler().fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)
val_data = scaler.transform(val_data)

In [ ]:
trainx = train_data.T
trainy = train_labels.reshape(-1,1).T

testx = test_data.T
testy =test_labels.reshape(-1,1).T

valx = val_data.T
valy = val_labels.reshape(-1,1).T

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape

((18, 611), (1, 611), (18, 127), (1, 127), (18, 108), (1, 108))

In [ ]:
X=trainx
Y=trainy

In [ ]:

shape_X = X.shape
shape_Y = Y.shape
n = X.shape[1]                   # training set size


print ('No. of training samples: ' + str(n))
print ('Number of features per sample: ' + str(shape_X[0]))

No. of training samples: 611
Number of features per sample: 18


In [ ]:
def model_architecture(X, Y):
   
    n_x = X.shape[0]                       # n_x -- the size of the input layer
    n_h = 100                              # n_h -- the size of the hidden layer
    n_y = len(yle.classes_)                                # n_y -- the size of the output layer
  
    return (n_x, n_h, n_y)

In [ ]:
def initialize_parameters(n_x, n_h, n_y):
    
    np.random.seed(2)

    W1 = np.random.randn(n_h,n_x) * 0.01                 # W1 -- weight matrix of shape (n_h, n_x)
    b1 =  np.zeros((n_h,1))                                # b1 -- bias vector of shape (n_h, 1)
    W2 = np.random.randn(n_y,n_h) * 0.01                  # W2 -- weight matrix of shape (n_y, n_h)
    b2 = np.zeros((n_y,1))                                 # b2 -- bias vector of shape (n_y, 1)
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
def tanh(x):
  return np.tanh(x)

def relu(x):
  return np.maximum(x,0)


def sigmoid(x):

    sigmoid = 1/(1 + np.exp(-x))
    return sigmoid 

In [ ]:
def softmax(x):
    expX = np.exp(x)
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [ ]:
def derivative_tanh(x):
  return (1 - np.power(x,2))

def derivative_relu(x):
  return np.array( x > 0, dtype = np.float32)

In [ ]:
def forward_propagation(X, parameters):
    
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']

    # Implement Forward Propagation to calculate A2 (probabilities)
    
    Z1 = np.dot(W1, X)+ b1                       # shape of Z1 (noOfHiddenNeurons,m)
    A1 = relu(Z1)                          # shape of A1 (noOfHiddenNeurons,m)
    Z2 = np.dot(W2,A1) + b2                      # shape of Z2 is (1,m)
    A2 = softmax(Z2)                          # shape of A2 is (1,m)

    assert(A2.shape == (4, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [ ]:
# Calculate the cost of our forward prop

def compute_cost(A2, y,parameters):
  n = Y.shape[1] # number of example
  cost = -(1 / m) * np.sum(y*np.log(A2))
  cost = float(np.squeeze(cost))
  assert(isinstance(cost, float))
  return cost



In [ ]:
def backprop(parameters, cache, X,  y):
    
    n = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    
    W1 = parameters['W1']
    W2 = parameters['W2']
    
        
    # Retrieve also A1 and A2 from dictionary "cache".
     
    A1 = cache['A1']
    A2 = cache['A2']
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 

    dZ2 = A2 -  y 
    dW2 = (1/n)*np.dot(A1.T, dZ2)
    db2 = np.sum(dZ2, axis=1, keepdims = True) / n
    dZ1 = (W2.T.dot(dZ2)) *  (derivative_relu(A1))
    dW1 = (dZ1.dot(X.T)) / n  
    db1 = np.sum(dZ1, axis=1, keepdims = True) / n
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [ ]:
def update(parameters, grads, learning_rate = 0.01):
    
    # Retrieve each parameter from the dictionary "parameters"
  
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    # Retrieve each gradient from the dictionary "grads"
    
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    
    # Update rule for each parameter
  
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2- learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
def NeuralNetwork(X, y, n_h, num_iterations = 10000, learning_rate = 0.01, print_cost=False):
   
    np.random.seed(3)
    n_x = model_architecture(X, Y)[0]
    n_y = model_architecture(X, Y)[2]
    
    # Initialize parameters
    
    parameters = initialize_parameters(n_x, n_h, n_y)
    W1 =  parameters['W1']                 # W1 -- weight matrix of shape (n_h, n_x)
    b1 =  parameters['b1']                 # b1 -- bias vector of shape (n_h, 1)
    W2 =  parameters['W2']                 # W2 -- weight matrix of shape (n_y, n_h)
    b2 =  parameters['b2']                 # b2 -- bias vector of shape (n_y, 1)
  
    # Loop (gradient descent)

    for i in range(0, num_iterations):

        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward_propagation(X, parameters)
        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost(A2, y, parameters)
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backprop(parameters, cache, X, y)
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update(parameters, grads)

        # Print the cost every 100 iterations
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

In [ ]:
def predict(parameters, X):
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.

    A2, cache = forward_propagation(X, parameters)
    predictions = A2.argmax
    
    return predictions

In [ ]:
# Build a model with a n_h-dimensional hidden layer
parameters = NeuralNetwork(X, y, n_h = 10, num_iterations = 10000, print_cost=True)

ValueError: ignored